In [1]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import seaborn as sns
from scipy.signal import butter,filtfilt,welch
from scipy import signal
import optuna
import numpy as np  
import optuna.visualization as vis
import plotly

ModuleNotFoundError: No module named 'numpy'

In [3]:
def Regular_1(L,Inh):
    L2=L*L
    Matrix=np.zeros((L2,L2),int)
    rng=np.random.default_rng(seed=1)
    for i in range(L):
        for j in range(L):
            ia=i
            ji=j
            id=i
            jd=j
            if ia+1==L:
                ia=-1
            if id==0:
                id=L
            if jd+1==L:
                jd=-1
            if ji==0:
                ji=L
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else :
                I=1
            Matrix[i*L+j][i*L+jd+1]=1*(2*I-1)## derecha I=0 -1,I=1 1
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else:
                I=1
            Matrix[i*L+j][i*L+ji-1]=-1*(2*I-1)## izquieda
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else:
                I=1
            Matrix[i*L+j][(ia+1)*L+j]=-1*(2*I-1)## arriba 
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else: 
                I=1
            Matrix[i*L+j][(id-1)*L+j]=-1*(2*I-1)# Abajo
    return Matrix

def small_word_Bi(L,P,Inh):
    Matrix=Regular_1(L,Inh)
    rng=np.random.default_rng(seed=1)
    for i in range(L*L):
        for j in range(L*L):
            if Matrix[i][j]==1 and rng.uniform(0.0, 1.0)<P:
                Matrix[i][j]==0
                Matrix[j][i]==0
                jj=rng.integers(0,L*L-1)
                if jj==i or jj==j:
                    jj=rng.integers(0,L*L-1)
                    Matrix[jj][i]=2
                    Matrix[i][jj]=2
                else:
                    Matrix[jj][i]=2
                    Matrix[i][jj]=2
            elif Matrix[i][j]==-1 and rng.uniform(0.0, 1.0)<P:
                Matrix[i][j]==0
                Matrix[j][i]==0
                jj=rng.integers(0,L*L-1)
                if jj==i or jj==j:
                    jj=rng.integers(0,L*L-1)
                    Matrix[jj][i]=-2
                    Matrix[i][jj]=-2
                else:
                    Matrix[jj][i]=-2
                    Matrix[i][jj]=-2
    for i in range(L*L):
        for j in range(L*L):
            if Matrix[i][j]==2:
                Matrix[i][j]=1
                Matrix[j][i]=1
            elif Matrix[i][j]==-2:
                Matrix[i][j]=-1
                Matrix[j][i]=-1
    return Matrix

def Random_bi(L,  P,  Inh):
    matrix=np.zeros((L*L,L*L),int)
    rng=np.random.default_rng(seed=1)
    for i in range(L*L):
        for j in range(L*L):
            I=rng.uniform(0.0,1.0)
            if I<Inh:
                I=0
            else:
                I=1
            if (matrix[i][j]!=1 or matrix[i][j]!=-1) and rng.uniform(0.0,1.0)<P:
            
                matrix[i][j]=-1*(2*I-1)
                matrix[j][i]=-1*(2*I-1)
    return matrix   
    

In [4]:
low_cutoff = 0.01  # Low cut-off frequency (Hz)
high_cutoff = 100.0  # High cut-off frequency (Hz)
order = 4  # Order of the filter
sampling_rate = 500 
b, a = signal.butter(order, [low_cutoff / (0.5 * sampling_rate), high_cutoff / (0.5 * sampling_rate)], btype='band')

In [5]:

fs = 500  # Sampling rate
nperseg = 256  # Segment length
noverlap = 128 # Overlap between segments
def Simulado_EGG(params):
    L, P, inhibidoras, trest, trelative, alpha, tmax, type_matrix = params
    # Compilación del programa
    resultado_compilacion = subprocess.run("g++ -O3 main.cpp Matrix_conect.cpp -o main", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if resultado_compilacion.returncode != 0:
        print("Error durante la compilación:")
        print(resultado_compilacion.stderr)
        return None
    
    # Ejecución del programa
    resultado_ejecucion = subprocess.run(f"./main {L} {P} {inhibidoras} {trest} {trelative} {alpha} {tmax} {type_matrix}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if resultado_ejecucion.returncode != 0:
        print("Error durante la ejecución:")
        print(resultado_ejecucion.stderr)
        return None
    
    # Procesamiento de los datos
    lines = resultado_ejecucion.stdout.strip().split('\n')
    data_list = [[float(num) for num in line.split()] for line in lines]
    data = np.array(data_list)[:,0]
    
    butter = signal.filtfilt(b, a, data[30:])
    frequencies, psd = signal.welch(butter, fs, nperseg=nperseg, noverlap=noverlap)
    frequencies=np.array(frequencies)
    welch_data = np.array([10 * np.log10(psd[i]**2) for i in range(len(frequencies))])
    
    delta = welch_data[np.logical_and(frequencies >= 1, frequencies <= 3)]
    theta = welch_data[np.logical_and(frequencies >= 4, frequencies <= 7)]
    alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 13)]
    beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

    return {
        'data': data,"butter": butter,"welch_data":welch,
        'delta':[delta.mean(),delta.std()],
        'theta':[theta.mean(), theta.std()],
        'alpha':[ alpha.mean(),alpha.std()],
        'beta': [beta.mean(), beta.std()]
    }



In [6]:
def Datos_reales(nombre):
    data = np.loadtxt(nombre)*1e6
    frequencies, psd = signal.welch(data, fs, nperseg=nperseg, noverlap=noverlap)
    frequencies=np.array(frequencies)
    welch_data = np.array([10 * np.log10(psd[i]**2) for i in range(len(frequencies))])
    
    delta = welch_data[np.logical_and(frequencies >= 1, frequencies <= 3)]
    theta = welch_data[np.logical_and(frequencies >= 4, frequencies <= 7)]
    alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 13)]
    beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

    return {
        'data': data,"butter": butter,"welch_data":welch,
        'delta':[delta.mean(),delta.std()],
        'theta':[theta.mean(), theta.std()],
        'alpha':[ alpha.mean(),alpha.std()],
        'beta': [beta.mean(), beta.std()]
    }


In [7]:
initial_params = [40, 0.01, 0.5, 1, 2, 0.1, 1000, 1]

generated_data = Simulado_EGG(initial_params)
reales_total= R = Datos_reales("F7_MNE.txt")


In [8]:
reales=reales_total["delta"]+reales_total["theta"]+reales_total["alpha"]+reales_total["beta"]

In [9]:

reales=reales_total["delta"]+reales_total["theta"]+reales_total["alpha"]+reales_total["beta"]

def objective(trial):
    params = np.zeros(8)
    
    params[0] = trial.suggest_int('L',10,100)
    params[1] = trial.suggest_float('P', 0.0, 1.0)
    params[2] = trial.suggest_float('Inh', 0.0, 1.0)
    params[3] = trial.suggest_float('Trest', -15.0, 15.0)
    params[4] = trial.suggest_float('Trelative',params[3],params[3]+10)  
    params[5] = trial.suggest_float('C_h', 0, 1.0)
    params[6] = 2030
    params[7]= 1
    #params[7] = trial.suggest_int('Tipo_red',0,2)

    
    generated_data_total = Simulado_EGG(params)
    if generated_data_total["delta"][1]-generated_data_total["delta"][1]==0:
        generated_data = generated_data_total["delta"] + generated_data_total["theta"] + generated_data_total["alpha"] + generated_data_total["beta"]
        difference = sum(abs(gd - rd) for gd, rd in zip(generated_data, reales))
        trial.report(difference, step=trial.number)
        
        # Prune si es necesario
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        return difference
    else:
        return float('inf') 
    # Calculating the difference between generated and real data
    

# Create and optimize the study
study = optuna.create_study(study_name='4bandas_SW_2000_sw', storage='sqlite:///Estudios/4bandas_SW_2000_sw.db')

# Continúa con la optimización del estudio
study.optimize(objective, n_trials=2000)


[I 2024-07-17 14:33:20,371] A new study created in RDB with name: 4bandas_SW_2000_sw
/tmp/ipykernel_4863/1334452019.py:28: RuntimeWarning: divide by zero encountered in log10
  welch_data = np.array([10 * np.log10(psd[i]**2) for i in range(len(frequencies))])
/home/juanmillan/.local/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
[I 2024-07-17 14:33:25,582] Trial 0 finished with value: inf and parameters: {'L': 32, 'P': 0.12538611585366088, 'Inh': 0.5365847424907445, 'Trest': 13.90714867796541, 'Trelative': 18.964116257186376, 'C_h': 0.8536606179695172}. Best is trial 0 with value: inf.
[I 2024-07-17 14:33:39,287] Trial 1 finished with value: inf and parameters: {'L': 54, 'P': 0.8647057558786954, 'Inh': 0.9288695453080862, 'Trest': 7.06486049291825, 'Trelative': 15.938716762180887, 'C_h': 0.19776714444384036}. Best is trial 0 with value: inf.
[I 2024-07-17 14:33:46,486] Trial 2 finished with 

In [10]:
print(study.study_name)
valid_trials = [t for t in study.trials if t.value is not None]
top_3_trials = sorted(valid_trials, key=lambda t: t.value)[:5]
for i, trial in enumerate(top_3_trials):
    print(f"\nTrial {i+1}:")
    print("Params:", trial.params)
    print("Value:", trial.value)
print("---------------------------------------","\n")
print("Best Result:", study.best_value)

vis.plot_optimization_history(study)


4bandas_SW_2000_sw

Trial 1:
Params: {'L': 63, 'P': 0.9613089140646414, 'Inh': 0.20650502150956862, 'Trest': -7.986730384904346, 'Trelative': -0.42223670612884345, 'C_h': 0.8159586128273556}
Value: 10.876353762210984

Trial 2:
Params: {'L': 53, 'P': 0.9046787009523168, 'Inh': 0.15778823775187678, 'Trest': -9.687620668785096, 'Trelative': -0.18116960530400308, 'C_h': 0.8197148249174167}
Value: 11.07240435614507

Trial 3:
Params: {'L': 55, 'P': 0.8826973844724002, 'Inh': 0.15910455760808478, 'Trest': -10.194511232909935, 'Trelative': -0.491989045009056, 'C_h': 0.8727623961623509}
Value: 11.256345458354922

Trial 4:
Params: {'L': 63, 'P': 0.9393046748528817, 'Inh': 0.14105080964675715, 'Trest': -10.190060115537962, 'Trelative': -0.43005605896150934, 'C_h': 0.9021432475056219}
Value: 11.594948367393755

Trial 5:
Params: {'L': 57, 'P': 0.9276378441376196, 'Inh': 0.15289363601402797, 'Trest': -7.047303337132972, 'Trelative': -0.10243484330872854, 'C_h': 0.9420765933491654}
Value: 11.68437592

In [11]:
vis.plot_param_importances(study)

[W 2024-07-17 17:52:36,014] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,015] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,016] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,017] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,018] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,019] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,021] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,022] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,023] Trial 23 is omitted in visualization because its objective value is inf or nan.


[W 2024-07-17 17:52:36,023] Trial 25 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,025] Trial 39 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,025] Trial 47 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,026] Trial 95 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,027] Trial 156 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,028] Trial 162 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,029] Trial 207 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,030] Trial 218 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,031] Trial 290 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:52:36,

In [12]:
vis.plot_parallel_coordinate(study)

[W 2024-07-17 17:54:08,070] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,070] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,071] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,072] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,072] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,073] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,073] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,074] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,075] Trial 23 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,075] Trial 2

In [13]:
optuna.visualization.plot_slice(study)

[W 2024-07-17 17:54:08,675] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,675] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,676] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,677] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,678] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,679] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,679] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,680] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,681] Trial 23 is omitted in visualization because its objective value is inf or nan.
[W 2024-07-17 17:54:08,682] Trial 2